# Creating a neural network from scratch

Big shoutout to the book which guides this project: "Neural Networks from Scratch in Python" by Harrison Kinsley & Daniel Kukieła

## Imports

In [16]:
import numpy as np
from nnfs.datasets import spiral_data

## Layer Classes

In [17]:
class Layer_Dense: 
    def __init__(self, n_input, n_neurons):
        self.weights = 0.01 * np.random.randn(n_input, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    def forward(self, inputs): 
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues): 
        # gradients on parameters for updating
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # gradients on values for next steps
        self.dinputs = np.dot(dvalues, self.weights.T)

## Activation Functions

In [18]:
class Activation_ReLU: 
    def forward(self, inputs): 
        self.inputs = inputs
        self.output = np.maximum(0, inputs)
    
    def backward(self, dvalues): 
        self.dinputs = dvalues.copy() # copy because of modifying 
        self.dinputs[self.inputs <= 0] = 0

In [19]:
class Activation_Softmax: 
    def forward(self, inputs):
        exp_value = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))  # subtracting max value for numerical stability => result will not change!
        self.output = exp_value / np.sum(exp_value, axis=1, keepdims=True)

    def backward(self, dvalues):
        # dinputs contains the gradiants
        # in softmax the result is a jacobian matrix 
        # =>    calculating the partial derivatives of every ouput of the softmax output with respect to each input seperately 
        # =>    at the end, every batch will be summed with the np.dot function to not return a 3D array but 2D array
        self.dinputs = np.empty_like(dvalues) 
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)): 
            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Calculate sample-wise gradient and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

In [20]:
class Activation_Sigmoid: 
    def forward(self, input): 
        pass

## Loss Functions

In [21]:
# parent class
class Loss: 
    def calculate(self, output, y):
        sample_loss = self.forward(output, y)
        data_loss = np.mean(sample_loss)
        return data_loss


In [22]:
class Loss_CategoricalCrossEntropy(Loss): 
    def forward(self, y_pred, y_true): 
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7) # to prevent 0 => log(0) is undefined

        """
        example: 
        
        y_pred_clipped = [[0.7, 0.1, 0.2],
                          [0.1, 0.5, 0.4],
                          [0.02, 0.9, 0.08]]
        """
        
        # single dimension => categorical labels
        if len(y_true.shape) == 1: # if is 1D-Array:
            # e.g. y_true = [1, 0, 0]
            correct_confidences = y_pred_clipped[range(samples), y_true]
    
        # 2 dimensions => one-hot encoded labels
        elif len(y_true.shape) == 2: 
            # e.g. y_true = [[0, 1, 0], [1, 0, 0], [1, 0, 0]]
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)            
        else: 
            raise Exception("Please use a different shape for y_true: shape = {0}".format(y_true.shape))
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods
    
    def backward(self, dvalues, y_true): 
        samples = len(dvalues) 
        labels = len(dvalues[0]) # number of labels in every sample

        # ensure one-hot encoding
        if len(y_true.shape) == 1: 
            y_true = np.eye(labels)[y_true]
        
        # calculate gradient 
        self.dinputs = -y_true / dvalues # derivative of loss function (-log(x)) => because of one-hot encoding the correct values get updated

        # normalize gradient => optimizers sum all the gradients before multiplying them with the learning rate 
        self.dinputs = self.dinputs / samples

In [23]:
# in case of using softmax and categorical corssentropy loss functions we can simplify 
# the backpropagation and therefore greatly enhance our performance 

class Activation_Softmax_Loss_CategoricalCrossentropy():
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossEntropy()
    
    def forward(self, inputs, y_true): 
        self.activation.forward(inputs)
        self.output = self.activation.output
        return self.loss.calculate(self.output, y_true)

    def backward(self, dvalues, y_true): 
        samples = len(dvalues)
        # if one-hot encoded turn into discrete values
        if len(y_true.shape) == 2: 
            y_true = np.argmax(y_true, axis=1) # axis 1: sample-wise performing 
        self.dinputs = dvalues.copy() # copy to not modify existing array
        # calc gradients
        self.dinputs[range(samples), y_true] -= 1
        # normalize gradients 
        self.dinputs /= samples  

## Optimizers

In [30]:
class Optimizer_SGD(): 
    def __init__(self, learning_rate=1.0, decay=0.0, momentum=0.0):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum

    def pre_update_params(self): 
        if self.decay: 
            self.current_learning_rate = self.learning_rate * (1 / (1 + self.decay * self.iterations))

    def update_params(self, layer):
        if self.momentum: 
            # if layer does not contain momentum arrays => create it
            if not hasattr(layer, "weight_momentums"):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases) # if weights do not have momentum array then also biases
            
            # build weight update with momentum
            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates # update momentum in layer for next update
            # same for biases
            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates
            
        # optimizing gradients without momentum
        else: 
            weight_updates = -self.current_learning_rate * layer.dweights 
            bias_updates = -self.current_learning_rate * layer.dbiases

        # update weights and biases
        layer.weights += weight_updates
        layer.biases += bias_updates

    def post_update_params(self): 
        self.iterations += 1

## Testing 

In [126]:
import matplotlib.pyplot as plt
def plot_progress(epoch, name, predict): 
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                        np.arange(y_min, y_max, 0.01))

    # Flatten the meshgrid points and make predictions
    grid_points = np.c_[xx.ravel(), yy.ravel()]
    Z = np.array([predict(x) for x in grid_points])
    Z = Z.reshape(xx.shape)

    colors = ['red', 'blue', 'green']


    # Use a colormap for both the scatter plot and contourf plot
    cmap = plt.cm.viridis # Choose the desired colormap

    # Plot the decision boundaries using filled contours with the specified colormap
    plt.contourf(xx, yy, Z, levels=np.arange(len(np.unique(y)) + 1) - 0.5, cmap=cmap, alpha=0.8)

    # Scatter plot for the original data points with the same colormap
    scatter = plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap, edgecolors='k', marker='.', s=40, alpha=0.8)

    # Create a legend
    legend_labels = np.unique(y)
    legend_handles = [plt.Line2D([0], [0], marker='o', color='w', label=label, markerfacecolor=cmap(label)) for label in legend_labels]
    plt.legend(handles=legend_handles, title='Classes')

    # Show the plot
    plt.savefig(f"img/{name}_{epoch}")


In [25]:
# calculate the time difference when using the optimized "activation_softmax_loss_categoricalcrossentropy" way
from timeit import timeit
import nnfs

nnfs.init()
softmax_outputs = np.array([[0.7, 0.1, 0.2],
                            [0.1, 0.5, 0.4],
                            [0.02, 0.9, 0.08]])
class_targets = np.array([0, 1, 1])

def f1():
    softmax_loss = Activation_Softmax_Loss_CategoricalCrossentropy()
    softmax_loss.backward(softmax_outputs, class_targets)
    dvalues1 = softmax_loss.dinputs
   
def f2():
    activation = Activation_Softmax()
    activation.output = softmax_outputs
    loss = Loss_CategoricalCrossEntropy()
    loss.backward(softmax_outputs, class_targets)
    activation.backward(loss.dinputs)
    dvalues2 = activation.dinputs
    


t1 = timeit(lambda: f1(), number=10000)
t2 = timeit(lambda: f2(), number=10000)

print("The optimized backward pass is {0} times faster!".format(t2/t1)) 

The optimized backward pass is 4.319570925691128 times faster!


In [128]:
# Create dataset    
X, y = spiral_data(samples=100, classes=3)

# Dense layer with 2 input features and 3 output values
dense1 = Layer_Dense(2, 64)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(64, 3)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
optimizer = Optimizer_SGD(decay=1e-3, momentum=0.9)


softmax = Activation_Softmax()

def predict_class(x): 
    dense1.forward(x)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    softmax.forward(dense2.output)
    return np.argmax(softmax.output, axis=1)


# loop the training process
epochs = 10001
for epoch in range(epochs): 

    # forward pass
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)

    loss = loss_activation.forward(dense2.output, y)
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions==y)
    
    # backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    if not epoch % 200:
        print(
            f'epoch: {epoch}, ' +
            f'acc: {accuracy:.3f}, ' +
            f'loss: {loss:.3f}', 
            f'lr: {optimizer.current_learning_rate}')
        plot_progress("spiral_data", epoch, predict_class)



    # update params based on calculated gradients
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()



epoch: 0, acc: 0.307, loss: 1.099 lr: 1.0
epoch: 100, acc: 0.480, loss: 1.029 lr: 0.9099181073703367
epoch: 200, acc: 0.477, loss: 1.021 lr: 0.8340283569641367
epoch: 300, acc: 0.697, loss: 0.738 lr: 0.7698229407236336
epoch: 400, acc: 0.787, loss: 0.528 lr: 0.7147962830593281
epoch: 500, acc: 0.863, loss: 0.379 lr: 0.66711140760507
epoch: 600, acc: 0.890, loss: 0.318 lr: 0.6253908692933083
epoch: 700, acc: 0.897, loss: 0.271 lr: 0.5885815185403178
epoch: 800, acc: 0.910, loss: 0.245 lr: 0.5558643690939411
epoch: 900, acc: 0.920, loss: 0.226 lr: 0.526592943654555
epoch: 1000, acc: 0.923, loss: 0.211 lr: 0.5002501250625312
epoch: 1100, acc: 0.933, loss: 0.198 lr: 0.4764173415912339
epoch: 1200, acc: 0.937, loss: 0.188 lr: 0.45475216007276037
epoch: 1300, acc: 0.940, loss: 0.179 lr: 0.43497172683775553
epoch: 1400, acc: 0.940, loss: 0.172 lr: 0.4168403501458941
epoch: 1500, acc: 0.940, loss: 0.166 lr: 0.4001600640256102
epoch: 1600, acc: 0.943, loss: 0.161 lr: 0.3847633705271258
epoch: 1

KeyboardInterrupt: 